# Issue 132_2 Notebook Toaletter nära SAT
* denna [Notebook](https://github.com/salgo60/Stockholm_Archipelago_Trail/blob/main/notebook/Issue_132_2_Notebook_Toaletter_n%C3%A4ra_SAT.ipynb), version 1 [Notebook](https://github.com/salgo60/Stockholm_Archipelago_Trail/blob/main/notebook/Issue_132_Notebook_Toaletter_n%C3%A4ra_SAT.ipynb)
* [Issue 132](https://github.com/salgo60/Stockholm_Archipelago_Trail/issues/132)
  

Se liknande lösning för Roslagsleden
* nu har vi SAT = wikidata [Q131318799](https://www.wikidata.org/wiki/Q131318799)
* "leden" sitter inte ihop utan varje ö har sitt segment

Output flyttat till ouput katalogen
* [SAT Dashboard](https://raw.githack.com/salgo60/Stockholm_Archipelago_Trail/main/notebook/output/SAT_ALL_IN_ONE_142_3_dashboard_latest.html)
   * [karta](https://raw.githack.com/salgo60/Stockholm_Archipelago_Trail/main/notebook/output/Issue_132_2_toaletter_nara_stockholm_archipelago_trail_latest.html) 


Tidigare
* [kartor/Issue_132_2_toaletter_nara_stockholm_archipelago_trail_2025_07_30_07_16.html](https://raw.githack.com/salgo60/Stockholm_Archipelago_Trail/main/kartor/Issue_132_2_toaletter_nara_stockholm_archipelago_trail_2025_07_30_07_16.html)
* [kartor/Issue_132_2_toaletter_nara_stockholm_archipelago_trail_2025_07_30_07_16.html](https://raw.githack.com/salgo60/Stockholm_Archipelago_Trail/main/kartor/Issue_132_2_toaletter_nara_stockholm_archipelago_trail_2025_07_30_07_16.html)
* [kartor/Issue_132_2_toaletter_nara_stockholm_archipelago_trail_2025_07_30_07_16.html](https://raw.githack.com/salgo60/Stockholm_Archipelago_Trail/main/kartor/Issue_132_2_toaletter_nara_stockholm_archipelago_trail_2025_07_30_07_16.html)

In [1]:
import time
from datetime import datetime

now = datetime.now()
timestamp = now.timestamp()
print(timestamp)  # Outputs seconds since Unix epoch


start_time = time.time()
print("Start:", datetime.now().strftime("%Y-%m-%d %H:%M:%S"))


1758621419.710435
Start: 2025-09-23 11:56:59


In [2]:
# -*- coding: utf-8 -*-
"""
Stockholm Archipelago Trail – toaletter nära leden
Omskriven helhet med:
- Sparar specifik fil (2025_07_30_07_21), "latest" och aktuell tidsstämpel
- Fler buffertlager (50 m, 100 m, 200 m) med färgkodning
- Ett enda lager för själva leden (inte ett lager per sektion)
- Separat lager för wheelchair=yes med egen ikon
- Popup med länkar till både OSM och Wheelmap
- About-box högst upp på kartan

OBS: Skriptet anropar Wikidata SPARQL och Overpass API (internet krävs).
"""

# !pip install geopandas shapely folium requests SPARQLWrapper --quiet

import os
from datetime import datetime
from collections import defaultdict
import html as _html
from string import Template

import requests
import geopandas as gpd
from shapely.geometry import LineString, MultiLineString, Point
from SPARQLWrapper import SPARQLWrapper, JSON
import folium
from folium import Marker, Icon, FeatureGroup, LayerControl, Popup

# =====================
# 0. Hjälpfunktioner
# =====================
def save_map_multiple(m: folium.Map):
    """Spara kartan till önskade filnamn.
    - Specifik tidsstämpel: ..._2025_07_30_07_21.html
    - Latest: ..._latest.html
    - Nuvarande tidsstämpel: ..._{YYYY_MM_DD_HH_MM}.html
    """
    os.makedirs("output", exist_ok=True)

    # b) Latest
    latest_path = (
        "output/Issue_132_2_toaletter_nara_stockholm_archipelago_trail_latest.html"
    )
    m.save(latest_path)

    # c) Även nuvarande tidsstämpel för spårbarhet
    timestamp = datetime.now().strftime("%Y_%m_%d_%H_%M")
    ts_path = f"output/Issue_132_2_toaletter_nara_stockholm_archipelago_trail_{timestamp}.html"
    m.save(ts_path)

    print(f"✅ Karta sparad:\n  • {latest_path}\n  • {ts_path}")


# =================== About-box (Notebook/Edge-safe) =========================
def add_about_box(
    m,
    issue_number: int,
    map_name: str,
    created_date: str | None = None,
    repo: str = "salgo60/Stockholm_Archipelago_Trail",
    collapsed: bool = False,
):
    """
    Notebook/Edge-säker About-box med string.Template ($-placeholder).
    """
    if created_date is None:
        created_date = datetime.now().strftime("%Y-%m-%d %H:%M")

    map_dom_id = m.get_name()
    box_id     = f"sat-about-{map_dom_id}"
    header_id  = f"{box_id}-hdr"
    issue_url  = f"https://github.com/{repo}/issues/{issue_number}"

    links = [
        ("SAT Dashboard", "https://raw.githack.com/salgo60/Stockholm_Archipelago_Trail/main/notebook/output/SAT_ALL_IN_ONE_142_3_dashboard_latest.html"),
        ("Project repo issues", "https://github.com/salgo60/Stockholm_Archipelago_Trail/issues?q=is%3Aissue"),
        ("Trail on OSM (rel 19012437)", "https://www.openstreetmap.org/relation/19012437"),
        ("Trail on Wikicommons", "https://commons.wikimedia.org/wiki/Category:Stockholm_Archipelago_Trail"),
        ("Official page", "https://stockholmarchipelagotrail.com/"),
        ("Unofficial FB group", "https://www.facebook.com/groups/2875020699552247"),
        ("Visit Sweden", "https://traveltrade.visitsweden.com/plan/news-sweden/Stockholm-Archipelago-Trail/"),
    ]
    links_html = "".join(
        f'<div><a href="{_html.escape(u)}" target="_blank" style="text-decoration:none;">🔗 {_html.escape(t)}</a></div>'
        for t, u in links
    )
    collapsed_class = "sat-about-collapsed" if collapsed else ""

    tpl = Template(r"""
<style>
  .sat-about { position: fixed; z-index: 10000; background: rgba(255,255,255,0.97);
    border: 2px solid #666; border-radius: 10px; box-shadow: 0 2px 6px rgba(0,0,0,0.25);
    font: 12px/1.35 -apple-system, system-ui, Segoe UI, Roboto, Helvetica, Arial, sans-serif;
    pointer-events: auto; min-width: 240px; max-width: 320px; }
  .sat-about-header { cursor: pointer; padding: 8px 10px; font-weight: 700;
    display: flex; align-items: center; gap: 6px; user-select: none;
    background: rgba(248,248,248,.9); border-bottom: 1px solid #e5e7eb; }
  .sat-about-body { padding: 8px 10px 10px 10px; }
  .sat-about-collapsed .sat-about-body { display: none; }
  .sat-chevron { margin-left: auto; transition: transform .15s ease-in-out; }
  .sat-about-collapsed .sat-chevron { transform: rotate(-90deg); }
  .sat-links { margin-top: 6px; padding-top: 6px; border-top: 1px solid #e5e7eb; }
  @media (max-width: 640px) { .sat-about { font-size: 11px; min-width: 200px; max-width: 260px; } }
</style>

<div id="$box_id" class="sat-about $collapsed_class">
  <div id="$header_id" class="sat-about-header" title="Click to collapse/expand">
    <span>ℹ️ About</span>
    <span class="sat-chevron">▸</span>
  </div>
  <div class="sat-about-body">
    <div style="font-weight:700;margin-bottom:4px;">Stockholm Archipelago Trail Map</div>
    <div>Issue: <a href="$issue_url" target="_blank">#$issue_number</a>&nbsp;&nbsp; Map: $map_name</div>
    <div>Created: $created_date</div>
    <div>Latest updates: saved as <i>_latest.html</i></div>
    <div class="sat-links">$links_html</div>
  </div>
</div>

<script>
(function(){
  var mapId = "$map_dom_id";
  var boxId = "$box_id";
  var hdrId = "$header_id";
  var storageKey = "satAboutCollapsed_" + "$map_dom_id" + "_#$issue_number";

  function setCollapsed(box, collapsed) {
    var body = box.querySelector(".sat-about-body");
    if (collapsed) { box.classList.add("sat-about-collapsed"); if (body) body.style.display = "none"; }
    else { box.classList.remove("sat-about-collapsed"); if (body) body.style.display = "block"; }
    try { localStorage.setItem(storageKey, collapsed ? "1" : "0"); } catch(e) {}
  }

  function placeBox(mapEl, box) {
    var zoom = mapEl.querySelector(".leaflet-control-zoom");
    var mr   = mapEl.getBoundingClientRect();
    var top = 10, left = 10;
    if (zoom) {
      var zr = zoom.getBoundingClientRect();
      top  = (zr.bottom - mr.top) + 8;
      left = (zr.left   - mr.left) + zr.width + 8;
    }
    box.style.top  = top  + "px";
    box.style.left = left + "px";
  }

  function init(tries) {
    tries = tries || 0;
    var mapEl = document.getElementById(mapId);
    var box   = document.getElementById(boxId);
    var hdr   = document.getElementById(hdrId);
    if (!mapEl || !box || !hdr) { if (tries < 60) return setTimeout(function(){ init(tries+1); }, 120); return; }

    try {
      var stored = localStorage.getItem(storageKey);
      if (stored === "1") setCollapsed(box, true);
      if (stored === "0") setCollapsed(box, false);
    } catch(e) {}

    hdr.addEventListener("click", function(e){
      e.stopPropagation();
      var willCollapse = !box.classList.contains("sat-about-collapsed");
      setCollapsed(box, willCollapse);
    });

    function doPlace(){ placeBox(mapEl, box); }
    var placeTries = 0, iv = setInterval(function(){ doPlace(); if (++placeTries > 25) clearInterval(iv); }, 120);
    window.addEventListener("resize", doPlace);
    requestAnimationFrame(doPlace);
  }

  if (document.readyState === "loading") { document.addEventListener("DOMContentLoaded", function(){ init(0); }); }
  else { init(0); }
})();
</script>
""")

    html_code = tpl.substitute(
        box_id=box_id,
        header_id=header_id,
        issue_number=issue_number,
        issue_url=issue_url,
        map_name=map_name,
        created_date=created_date,
        links_html=links_html,
        collapsed_class=collapsed_class,
        map_dom_id=map_dom_id,
    )

    m.get_root().html.add_child(folium.Element(html_code))


# =====================
# 1. Hämta SAT-etapper från Wikidata
# =====================
print("🔍 Hämtar SAT-etapper från Wikidata…")
sparql = SPARQLWrapper("https://query.wikidata.org/sparql")
sparql.setQuery(
    """
    SELECT ?item ?itemLabel ?islandLabel ?osmid WHERE {
      ?item wdt:P361 wd:Q131318799;
            wdt:P31 wd:Q2143825;
            wdt:P402 ?osmid.
      OPTIONAL { ?item wdt:P706 ?island. }
      SERVICE wikibase:label { bd:serviceParam wikibase:language "sv,en". }
    }
    """
)
sparql.setReturnFormat(JSON)
results = sparql.query().convert()

etapper = []
for r in results["results"]["bindings"]:
    etapper.append(
        {
            "id": r["osmid"]["value"],
            "label": r.get("itemLabel", {}).get("value", ""),
            "island": r.get("islandLabel", {}).get("value", ""),
        }
    )

# =====================
# 2. Läs SAT-geometrier från lokal GeoJSON (utan properties)
# =====================
from shapely.geometry import MultiLineString

geojson_path = "SAT_full.geojson"  # eller "/mnt/data/SAT_full.geojson"
print(f"📁 Läser SAT-geometrier från {geojson_path} …")

gdf_sat = gpd.read_file(geojson_path)

# Säkerställ CRS = WGS84
if gdf_sat.crs is None:
    gdf_sat = gdf_sat.set_crs("EPSG:4326", allow_override=True)
else:
    gdf_sat = gdf_sat.to_crs("EPSG:4326")

# Behåll bara linjer
gdf_lines = gdf_sat[gdf_sat.geometry.geom_type.isin(["LineString", "MultiLineString"])].copy()
if gdf_lines.empty:
    raise SystemExit("❌ Inga LineString/MultiLineString i SAT_full.geojson – kontrollera filen.")

# a) gdf_trail: alla linjedelar (för buffert m.m.)
trail_parts = gdf_lines.explode(index_parts=False, ignore_index=True)
gdf_trail = trail_parts[["geometry"]].copy()
gdf_trail.set_crs("EPSG:4326", inplace=True)

# b) meta_gdf: ett enda MultiLineString-objekt som representerar hela leden
#    (används i sjoin_nearest så att popup får något att joina mot)
all_lines = list(trail_parts.geometry.values)
multi = MultiLineString([ls for ls in all_lines])
meta_gdf = gpd.GeoDataFrame(
    [{"label": "Stockholm Archipelago Trail", "island": None, "geometry": multi}],
    crs="EPSG:4326"
)

print(f"✅ Läste in {len(gdf_trail)} linjedelar och skapade ett MultiLineString för hela leden.")

# =====================
# 3. Skapa buffertar (50 m, 100 m, 200 m)
# =====================
print("🧮 Skapar buffertar 50/100/200 m…")
trail_utm = gdf_trail.to_crs(3006)

buffer50_utm = trail_utm.buffer(50)
buffer100_utm = trail_utm.buffer(100)
buffer200_utm = trail_utm.buffer(200)

buffer50 = buffer50_utm.to_crs(4326)
buffer100 = buffer100_utm.to_crs(4326)
buffer200 = buffer200_utm.to_crs(4326)

# =====================
# 4. Hämta toaletter (robust: speglar, retries, tiles, cache)
# =====================
import time, json, math
from pathlib import Path
from shapely.geometry import Polygon
from shapely.ops import unary_union

overpass_endpoints = [
    "https://overpass-api.de/api/interpreter",
    "https://z.overpass-api.de/api/interpreter",
    "https://overpass.kumi.systems/api/interpreter",
    # "https://overpass.openstreetmap.ru/api/interpreter",  # ibland opålitlig, slå på vid behov
]

HEADERS = {"User-Agent": "SAT-toilet-map/1.0 (contact: your-email@example.com)"}
CACHE_PATH = Path("output/toilets_cache.geojson")
os.makedirs("output", exist_ok=True)

def bbox_to_tiles(bbox, max_deg=0.25):
    """
    Dela upp bbox (minx, miny, maxx, maxy) i mindre rutor max max_deg grader per sida.
    ~0.25° är lagom för att undvika för stora svar.
    """
    minx, miny, maxx, maxy = bbox
    xs = list(frange(minx, maxx, max_deg))
    ys = list(frange(midy:=miny, maxy, max_deg))
    tiles = []
    for x1 in xs:
        x2 = min(x1 + max_deg, maxx)
        for y1 in ys:
            y2 = min(y1 + max_deg, maxy)
            tiles.append((x1, y1, x2, y2))
    return tiles

def frange(start, stop, step):
    vals, cur = [], start
    # undvik oändlig loop pga float
    while cur < stop - 1e-9:
        vals.append(cur)
        cur += step
    return vals

def overpass_fetch(query, timeout=120, retries=3, backoff=2.0):
    """
    Kör samma query mot flera endpoints, med retries/backoff.
    Returnerar första lyckade response.json().
    Kastar Exception om alla försök misslyckas.
    """
    last_err = None
    for ep in overpass_endpoints:
        for attempt in range(1, retries+1):
            try:
                r = requests.post(ep, data={"data": query}, headers=HEADERS, timeout=timeout)
                if r.status_code == 200:
                    return r.json()
                # 429/5xx → vänta och försök igen/byte endpoint
                time.sleep(backoff * attempt)
            except Exception as e:
                last_err = e
                time.sleep(backoff * attempt)
        # prova nästa endpoint
    raise RuntimeError(f"Overpass misslyckades på alla speglar. Sista fel: {last_err}")

def build_query(overpass_bbox):
    return f"""
[out:json][timeout:90];
(
  node["amenity"="toilets"]{overpass_bbox};
  way["amenity"="toilets"]{overpass_bbox};
  relation["amenity"="toilets"]{overpass_bbox};
);
out center tags bb;
"""

def parse_elements_to_points(elements):
    rows = []
    for el in elements:
        t = el.get("type")
        tags = el.get("tags", {}) or {}
        el_id = el.get("id")

        if t == "node" and "lat" in el and "lon" in el:
            pt = Point(el["lon"], el["lat"])
            rows.append({"geometry": pt, "tags": tags, "id": el_id, "elem_type": t})
        elif t in ("way", "relation"):
            c = el.get("center")
            if c and "lat" in c and "lon" in c:
                pt = Point(c["lon"], c["lat"])
                rows.append({"geometry": pt, "tags": tags, "id": el_id, "elem_type": t})
            elif "geometry" in el and isinstance(el["geometry"], list) and len(el["geometry"]) >= 3:
                coords = [(p["lon"], p["lat"]) for p in el["geometry"] if "lon" in p and "lat" in p]
                if coords:
                    try:
                        # polygon om sluten, annars line → centroid
                        from shapely.geometry import Polygon, LineString
                        geom = Polygon(coords) if len(coords) >= 4 and coords[0] == coords[-1] else LineString(coords)
                        pt = geom.centroid
                        rows.append({"geometry": pt, "tags": tags, "id": el_id, "elem_type": t})
                    except Exception:
                        pass
    return rows

# 4a) Bygg tiles över trailens bbox
bbox = gdf_trail.total_bounds  # (minx, miny, maxx, maxy)
tiles = bbox_to_tiles(bbox, max_deg=0.25)
print(f"🧩 Delar upp bbox i {len(tiles)} rutor för stabilare Overpass-svar…")

# 4b) Försök hämta från Overpass; om allt kraschar, läs cache om den finns
toilet_rows = []
try:
    for i, (minx, miny, maxx, maxy) in enumerate(tiles, start=1):
        overpass_bbox = f"({miny},{minx},{maxy},{maxx})"
        q = build_query(overpass_bbox)
        print(f"📡 Tile {i}/{len(tiles)} … ", end="", flush=True)
        data = overpass_fetch(q, timeout=120, retries=3, backoff=2.0)
        els = data.get("elements", [])
        rows = parse_elements_to_points(els)
        toilet_rows.extend(rows)
        print(f"{len(rows)} objekt")
except Exception as e:
    if CACHE_PATH.exists():
        print(f"⚠️ Overpass nere — använder cache: {CACHE_PATH}")
        gdf_toilets = gpd.read_file(CACHE_PATH)
    else:
        raise SystemExit("❌ Kunde inte hämta toaletter och ingen cache fanns. Kör igen senare eller ange en lokal källa.") from e
else:
    # 4c) Bygg GeoDataFrame och spara cache
    gdf_toilets = gpd.GeoDataFrame(toilet_rows, crs="EPSG:4326")
    # Rensa ev. dubbletter (samma id/elem_type)
    if not gdf_toilets.empty:
        gdf_toilets["uid"] = gdf_toilets["elem_type"].astype(str) + ":" + gdf_toilets["id"].astype(str)
        gdf_toilets = gdf_toilets.drop_duplicates(subset=["uid"]).drop(columns=["uid"])
    print(f"✅ Hittade totalt {len(gdf_toilets)} toaletter (node/way/relation). Sparar cache…")
    try:
        gdf_toilets.to_file(CACHE_PATH, driver="GeoJSON")
        print(f"💾 Cache sparad: {CACHE_PATH}")
    except Exception as e:
        print(f"⚠️ Kunde inte spara cache: {e}")
# =====================
# 5. Filtrera toaletter inom 200 m (robust)
# =====================
buffer200_union = buffer200.union_all()
in_range200 = gdf_toilets[gdf_toilets.intersects(buffer200_union)]
print(f"✅ {len(in_range200)} toaletter ligger inom 200 m från leden")

# =====================
# 6. Närmaste etapp för metadata i popup
# =====================
print("🔗 Matchar toaletter till närmaste etapp…")

toilets_utm = in_range200.to_crs(3006)
meta_utm = meta_gdf.to_crs(3006)
joined = gpd.sjoin_nearest(
    toilets_utm, meta_utm[["label", "island", "geometry"]], how="left", distance_col="distance_m"
).to_crs(4326)

# =====================
# 7. Bygg interaktiv karta
# =====================
print("🗺️ Bygger karta…")

# Starta vid centroid av första geometri
center = [gdf_trail.geometry.iloc[0].centroid.y, gdf_trail.geometry.iloc[0].centroid.x]
m = folium.Map(location=center, zoom_start=9)

# a) Etappgeometri – ett enda lager
folium.GeoJson(
    meta_gdf.union_all(),
    name="SAT-leden",
    tooltip="SAT-leden",
    style_function=lambda x: {"color": "blue", "weight": 3},
).add_to(m)

# b) Buffertlager (50/100/200 m) – färgkodade
folium.GeoJson(
    buffer50.union_all(),
    name="Buffert 50 m",
    style_function=lambda x: {"fillColor": "#2ECC71", "color": "#2ECC71", "weight": 1, "fillOpacity": 0.12},
).add_to(m)

folium.GeoJson(
    buffer100.union_all(),
    name="Buffert 100 m",
    style_function=lambda x: {"fillColor": "#F39C12", "color": "#F39C12", "weight": 1, "fillOpacity": 0.12},
).add_to(m)

folium.GeoJson(
    buffer200.union_all(),
    name="Buffert 200 m",
    style_function=lambda x: {"fillColor": "#E74C3C", "color": "#E74C3C", "weight": 1, "fillOpacity": 0.12},
).add_to(m)

# c) Toalettmarkörer (inkl. särskilt lager för wheelchair=yes)
wheel_group = FeatureGroup(name="Toaletter: wheelchair=yes (inom 200 m)")
good_group = FeatureGroup(name="Godkända toaletter (inom 200 m)")
warn_group = FeatureGroup(name="Varningar / Ofullständiga (inom 200 m)")

for _, row in joined.iterrows():

    tags = row["tags"]
    el_id = row["id"]
    osm_url = f"https://www.openstreetmap.org/node/{el_id}"
    wheelmap_url = f"https://wheelmap.org/node/{el_id}"

    popup_text = (
        f"<b><a href='{osm_url}' target='_blank'>OSM objekt</a></b> · "
        f"<a href='{wheelmap_url}' target='_blank'>Wheelmap</a><br>"
    )

    # Metadata
    if "image" in tags:
        popup_text += f"<img src='{tags['image']}' width='200'><br>"
    if "opening_hours" in tags:
        popup_text += f"<b>Öppettider:</b> {tags['opening_hours']}<br>"
    if "wheelchair" in tags:
        popup_text += f"<b>Rullstol:</b> {tags['wheelchair']}<br>"
    if "toilets:disposal" in tags:
        popup_text += f"<b>Avfallshantering:</b> {tags['toilets:disposal']}<br>"
    if "drinking_water" in tags:
        popup_text += f"<b>Dricksvatten:</b> {tags['drinking_water']}<br>"
    popup_text += "<hr style='margin:5px 0;'><ul style='padding-left:18px;'>"
    # --- Alias/fält-normalisering till popup-kollen ---
    # Papper: acceptera både toilets:paper_supply & toilets:paper_supplied
    paper_keys = ("toilets:paper_supply", "toilets:paper_supplied")
    has_paper = any(k in tags for k in paper_keys)

    required = ["amenity", "access", "unisex", "toilets:disposal", "wheelchair"]
    recommended = [
        "toilets:handwashing",   # valfritt men bra
        "opening_hours",
        "fee",
        "operator",
        "description",
        "drinking_water",        # bra att visa om det finns
    ]
    
    missing_req = [t for t in required if t not in tags]
    missing_opt = [t for t in recommended if t not in tags]
    
    if not has_paper:
        missing_opt.append("toilets:paper_supply / toilets:paper_supplied")


    for t in missing_req:
        popup_text += f"<li><b>{t}</b> saknas</li>"
    for t in missing_opt:
        popup_text += f"<li>{t} saknas</li>"
    popup_text += "</ul><i>Tips: kontrollera viktiga attribut</i>"

    # Lagerlogik
    is_wheel_yes = tags.get("wheelchair") == "yes"
    if is_wheel_yes:
        # Särskild ikon/lager för wheelchair=yes
        Marker(
            location=[row.geometry.y, row.geometry.x],
            popup=Popup(popup_text, max_width=300),
            icon=Icon(color="green", icon="wheelchair", prefix="fa"),
        ).add_to(wheel_group)
        continue

    # Övriga markörer i godkända/varningar
    icon_color = "red" if missing_req else "blue"
    Marker(
        location=[row.geometry.y, row.geometry.x],
        popup=Popup(popup_text, max_width=300),
        icon=Icon(color=icon_color, icon="info-sign"),
    ).add_to(warn_group if missing_req else good_group)

m.add_child(wheel_group)
m.add_child(good_group)
m.add_child(warn_group)
LayerControl(collapsed=False).add_to(m)

# =====================
# 8. Spara
# =====================
add_about_box(m, issue_number=132, map_name="Toaletter nära leden")
save_map_multiple(m)


🔍 Hämtar SAT-etapper från Wikidata…
📁 Läser SAT-geometrier från SAT_full.geojson …
✅ Läste in 788 linjedelar och skapade ett MultiLineString för hela leden.
🧮 Skapar buffertar 50/100/200 m…
🧩 Delar upp bbox i 30 rutor för stabilare Overpass-svar…
📡 Tile 1/30 … 24 objekt
📡 Tile 2/30 … 19 objekt
📡 Tile 3/30 … 218 objekt
📡 Tile 4/30 … 31 objekt
📡 Tile 5/30 … 4 objekt
📡 Tile 6/30 … 24 objekt
📡 Tile 7/30 … 28 objekt
📡 Tile 8/30 … 66 objekt
📡 Tile 9/30 … 14 objekt
📡 Tile 10/30 … 4 objekt
📡 Tile 11/30 … 4 objekt
📡 Tile 12/30 … 27 objekt
📡 Tile 13/30 … 26 objekt
📡 Tile 14/30 … 7 objekt
📡 Tile 15/30 … 1 objekt
📡 Tile 16/30 … 0 objekt
📡 Tile 17/30 … 10 objekt
📡 Tile 18/30 … 37 objekt
📡 Tile 19/30 … 15 objekt
📡 Tile 20/30 … 9 objekt
📡 Tile 21/30 … 0 objekt
📡 Tile 22/30 … 0 objekt
📡 Tile 23/30 … 17 objekt
📡 Tile 24/30 … 5 objekt
📡 Tile 25/30 … 5 objekt
📡 Tile 26/30 … 0 objekt
📡 Tile 27/30 … 0 objekt
📡 Tile 28/30 … 2 objekt
📡 Tile 29/30 … 0 objekt
📡 Tile 30/30 … 8 objekt
✅ Hittade totalt 605 toalet

In [3]:
 # End timer and calculate duration
end_time = time.time()
elapsed_time = end_time - start_time

# Print current date and total time
print("Date:", datetime.now().strftime("%Y-%m-%d %H:%M:%S"))
print("Total time elapsed: {:.2f} seconds".format(elapsed_time))

Date: 2025-09-23 11:58:12
Total time elapsed: 72.35 seconds
